### 221_event_openinghours 

* [Issue 221](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/221)

In [1]:
import time
import datetime  
start_time = time.time()
start_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started: {start_str}")


Started: 2025-10-04 14:55


In [2]:
import requests
import json
import time
import feedparser

WD_SPARQL_ENDPOINT = "https://query.wikidata.org/sparql"

def fetch_sat_objects():
    query = """
    SELECT ?item ?itemLabel ?website ?commonscat ?svwiki
    WHERE {
      ?item wdt:P6104 wd:Q134294510 .
      OPTIONAL { ?item wdt:P856 ?website }
      OPTIONAL { ?item wdt:P373 ?commonscat }
      OPTIONAL {
        ?svwiki schema:about ?item ;
                schema:isPartOf <https://sv.wikipedia.org/> .
      }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
    }
    """
    r = requests.get(WD_SPARQL_ENDPOINT, params={"query": query, "format": "json"}, headers={"User-Agent":"SAT-Watcher/1.0"})
    r.raise_for_status()
    data = r.json()
    items = []
    for b in data["results"]["bindings"]:
        items.append({
            "qid": b["item"]["value"].split("/")[-1],
            "label": b.get("itemLabel", {}).get("value"),
            "website": b.get("website", {}).get("value"),
            "commonscat": b.get("commonscat", {}).get("value"),
            "svwiki": b.get("svwiki", {}).get("value")
        })
    return items


def check_website(website_url):
    """Try to detect if a website has RSS or last-modified"""
    try:
        r = requests.get(website_url, timeout=10, headers={"User-Agent":"SAT-Watcher/1.0"})
        lastmod = r.headers.get("Last-Modified")

        # crude RSS check
        if "xml" in r.headers.get("Content-Type",""):
            return {"status": "rss-feed", "lastmod": lastmod}
        if "<rss" in r.text.lower():
            return {"status": "rss-feed-embedded", "lastmod": lastmod}
        if lastmod:
            return {"status": "ok", "lastmod": lastmod}
        return {"status": "ok"}
    except Exception as e:
        return {"status": "error", "error": str(e)}


def build_activity_feed():
    sat_items = fetch_sat_objects()
    feed = []

    for it in sat_items:
        entry = {"qid": it["qid"], "label": it["label"]}
        if it.get("website"):
            entry["website_check"] = check_website(it["website"])
        if it.get("commonscat"):
            entry["commons_url"] = f"https://commons.wikimedia.org/wiki/Category:{it['commonscat']}"
        if it.get("svwiki"):
            entry["svwiki"] = it["svwiki"]
        feed.append(entry)

    # save snapshot
    with open("sat_activity_feed.json", "w", encoding="utf-8") as f:
        json.dump(feed, f, ensure_ascii=False, indent=2)

    return feed


if __name__ == "__main__":
    print("Building activity feed…")
    data = build_activity_feed()
    print(json.dumps(data, indent=2, ensure_ascii=False))


Building activity feed…
[
  {
    "qid": "Q294787",
    "label": "Öja",
    "commons_url": "https://commons.wikimedia.org/wiki/Category:Öja (Stockholm archipelago)"
  },
  {
    "qid": "Q28375391",
    "label": "Yxlans fyr",
    "commons_url": "https://commons.wikimedia.org/wiki/Category:Yxlan, lighthouse"
  },
  {
    "qid": "Q28934440",
    "label": "Björkö-Arholma sjömannaförening",
    "website_check": {
      "status": "ok",
      "lastmod": "Fri, 26 Sep 2025 11:16:06 GMT"
    },
    "commons_url": "https://commons.wikimedia.org/wiki/Category:Björkö-Arholma Sjömannaförening"
  },
  {
    "qid": "Q28934474",
    "label": "Sandhamns museum",
    "website_check": {
      "status": "ok"
    },
    "commons_url": "https://commons.wikimedia.org/wiki/Category:Sandhamns museum"
  },
  {
    "qid": "Q29528279",
    "label": "Pestkyrkogård, Öja-Landsort",
    "commons_url": "https://commons.wikimedia.org/wiki/Category:Öja- Landsort Pestkyrkogård"
  },
  {
    "qid": "Q30315439",
    "label"

In [7]:
import os, html, requests, folium, datetime
from jinja2 import Template
from datetime import datetime as dt, timedelta
from datetime import datetime, timedelta, timezone

# ========================
# Config
# ========================
MAP_CENTER = [59.5, 18.8]
MAP_ZOOM   = 7
OUTPUT_PREFIX = "SAT_event_openinghours_tracker"

HEADERS = {
    "User-Agent": (
        "SAT_event_openinghours_tracker/1.0 "
        "(https://github.com/salgo60/Stockholm_Archipelago_Trail; salgo60@example.com)"
    )
}

# ========================
# Commons latest uploads
# ========================
def fetch_depicts(title):
    """Get depicts (P180) Wikidata IDs for a Commons file"""
    url = "https://commons.wikimedia.org/w/api.php"
    params = {
        "action": "wbgetentities",
        "sites": "commonswiki",
        "titles": title,
        "props": "claims",
        "format": "json"
    }
    r = requests.get(url, params=params, headers=HEADERS)
    r.raise_for_status()
    depicts_data = r.json()
    depicts = []
    for entity in depicts_data.get("entities", {}).values():
        for pid, claims in entity.get("claims", {}).items():
            if pid == "P180":
                for c in claims:
                    if "mainsnak" in c and "datavalue" in c["mainsnak"]:
                        depicts.append(c["mainsnak"]["datavalue"]["value"]["id"])
    return depicts

def get_coords_from_wikidata(qid):
    url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
    r = requests.get(url, headers=HEADERS)
    if r.status_code != 200:
        return None
    entity = r.json()["entities"].get(qid, {})
    coords = entity.get("claims", {}).get("P625")
    if not coords:
        return None
    val = coords[0]["mainsnak"]["datavalue"]["value"]
    return val["latitude"], val["longitude"]

def fetch_commons_latest(category="Stockholm_Archipelago_Trail", limit=50):
    url = "https://commons.wikimedia.org/w/api.php"
    params = {
        "action": "query",
        "generator": "categorymembers",
        "gcmtitle": f"Category:{category}",
        "gcmnamespace": "6",
        "gcmlimit": limit,
        "gcmsort": "timestamp",
        "gcmdir": "desc",
        "prop": "imageinfo",
        "iiprop": "url|timestamp|user",
        "format": "json",
        "formatversion": 2
    }
    r = requests.get(url, params=params, headers=HEADERS)
    r.raise_for_status()
    data = r.json()
    files = {}
    for page in data.get("query", {}).get("pages", []):
        title = page["title"]
        info  = page["imageinfo"][0]
        files[title] = {
            "title": title,
            "url": info["url"],
            "timestamp": info["timestamp"],
            "user": info["user"],
            "depicts": fetch_depicts(title)
        }
    return files

def add_commons_latest_layer(m, category="Stockholm_Archipelago_Trail", limit=50):
    fg = folium.FeatureGroup(name="📷 Latest Commons SAT uploads", show=True)
    files = fetch_commons_latest(category, limit)
    for f in files.values():
        latlon = None
        for qid in f["depicts"]:
            coords = get_coords_from_wikidata(qid)
            if coords:
                latlon = coords
                break

        popup_html = f"""
        <div style="min-width:220px;font-size:13px;">
          <b>{html.escape(f['title'])}</b><br>
          <img src="{f['url']}" width="200"><br>
          📤 {html.escape(f['user'])}<br>
          🕒 {f['timestamp']}<br>
          <a href="{f['url']}" target="_blank">Open on Commons</a>
        </div>
        """

        folium.Marker(
            latlon if latlon else MAP_CENTER,
            icon=folium.Icon(color="blue" if latlon else "gray", icon="camera", prefix="fa"),
            popup=folium.Popup(popup_html, max_width=250)
        ).add_to(fg)

    fg.add_to(m)

# ========================
# Wikidata recent edits
# ========================
def fetch_recent_wikidata_edits(days=1):
    cutoff = (datetime.now(timezone.utc) - timedelta(days=days)).strftime("%Y-%m-%dT%H:%M:%SZ")
    query = f"""
    SELECT ?item ?coord ?modified WHERE {{
      ?item schema:dateModified ?modified ;
            wdt:P625 ?coord .
      FILTER (?modified > "{cutoff}"^^xsd:dateTime)
    }}
    LIMIT 200
    """
    r = requests.get("https://query.wikidata.org/sparql",
                     params={"query": query, "format": "json"},
                     headers=HEADERS, timeout=60)
    r.raise_for_status()
    data = r.json()
    edits = []
    for b in data["results"]["bindings"]:
        coord_val = b["coord"]["value"]
        # Ensure it's a WKT literal ("Point(" ...)
        if not coord_val.startswith("Point("):
            continue
        coord = coord_val.replace("Point(", "").replace(")", "")
        try:
            lon, lat = map(float, coord.split())
        except Exception:
            continue
        qid = b["item"]["value"].split("/")[-1]
        edits.append({
            "qid": qid,
            "lat": lat,
            "lon": lon,
            "modified": b["modified"]["value"]
        })
    return edits



def add_recent_edits_layer(m, days=1):
    edits = fetch_recent_wikidata_edits(days=days)
    fg = folium.FeatureGroup(name=f"🟢 Wikidata edits (last {days} days)", show=True)
    for e in edits:
        popup_html = f"""
        <div style="min-width:220px;font-size:13px;">
          <b>{html.escape(e['label'] or e['qid'])}</b><br>
          <i>{html.escape(e['desc'])}</i><br>
          🕒 {e['modified']}<br>
          <a href="https://www.wikidata.org/wiki/{e['qid']}" target="_blank">🔗 Wikidata</a> |
          <a href="https://www.wikidata.org/w/index.php?title={e['qid']}&action=history" target="_blank">📜 History</a>
        </div>
        """
        folium.CircleMarker(
            [e["lat"], e["lon"]],
            radius=6,
            color="green",
            fill=True,
            fill_opacity=0.7,
            popup=folium.Popup(popup_html, max_width=300)
        ).add_to(fg)
    fg.add_to(m)

# ========================
# About box
# ========================
def add_about_box(
    m,
    issue_number: int,
    map_name: str,
    created_date: str | None = None,
    repo: str = "salgo60/Stockholm_Archipelago_Trail",
    collapsed: bool = False,
    offset_px=(10, 54),
):
    if created_date is None:
        created_date = dt.now().strftime("%Y-%m-%d %H:%M")

    map_dom_id = m.get_name()
    box_id     = f"sat-about-{map_dom_id}"
    header_id  = f"{box_id}-hdr"
    issue_url  = f"https://github.com/{repo}/issues/{issue_number}"
    top, left  = offset_px
    collapsed_class = "sat-about-collapsed" if collapsed else ""

    links = [
        ("SAT Dashboard", "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_dashboard_latest.html"),
        ("Project repo issues", "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue"),
        ("Trail on OSM (rel 19012437)", "https://www.openstreetmap.org/relation/19012437"),
        ("Trail on Wikicommons", "https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail"),
        ("Official page", "https://stockholmarchipelagotrail.com/"),
        ("Unofficial FB group", "https://www.facebook.com/groups/2875020699552247"),
        ("Visit Sweden", "https://traveltrade.visitsweden.com/plan/news-sweden/Stockholm-Archipelago-Trail/"),
    ]
    links_html = "".join(
        f'<div><a href="{html.escape(u)}" target="_blank" style="text-decoration:none;">🔗 {html.escape(t)}</a></div>'
        for t, u in links
    )

    tpl = Template("""<style>
  .sat-about {
    position: fixed; z-index: 10000;
    top: {{top}}px; left: {{left}}px;
    background: rgba(255,255,255,0.97);
    border: 2px solid #666; border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.25);
    font: 12px/1.35 -apple-system, system-ui, Segoe UI, Roboto, Helvetica, Arial, sans-serif;
    min-width: 240px; max-width: 320px; pointer-events: auto;
  }
  .sat-about-header { cursor: pointer; padding: 8px 10px; font-weight: 700;
    display: flex; align-items: center; gap: 6px; user-select: none;
    background: rgba(248,248,248,.9); border-bottom: 1px solid #e5e7eb; }
  .sat-about-body { padding: 8px 10px 10px 10px; }
  .sat-about-collapsed .sat-about-body { display: none; }
  .sat-chevron { margin-left: auto; transition: transform .15s ease-in-out; }
  .sat-about-collapsed .sat-chevron { transform: rotate(-90deg); }
  .sat-links { margin-top: 6px; padding-top: 6px; border-top: 1px solid #e5e7eb; }
</style>

<div id="{{box_id}}" class="sat-about {{collapsed_class}}">
  <div id="{{header_id}}" class="sat-about-header" title="Click to collapse/expand">
    <span>ℹ️ About</span><span class="sat-chevron">▸</span>
  </div>
  <div class="sat-about-body">
    <div style="font-weight:700;margin-bottom:4px;">Stockholm Archipelago Trail Map</div>
    <div>Issue: <a href="{{issue_url}}" target="_blank">#{{issue_number}}</a>&nbsp;&nbsp; Map: {{map_name}}</div>
    <div>Created: {{created_date}}</div>
    <div>Latest updates: saved as <i>_latest.html</i></div>
    <div class="sat-links">{{links_html}}</div>
  </div>
</div>
<script>(function(){var boxId="{{box_id}}",hdrId="{{header_id}}",
storageKey="satAboutCollapsed_{{map_dom_id}}_#{{issue_number}}";
function setCollapsed(b,c){if(!b)return;if(c)b.classList.add("sat-about-collapsed");
else b.classList.remove("sat-about-collapsed");
try{localStorage.setItem(storageKey,c?"1":"0");}catch(e){}}
function init(){var b=document.getElementById(boxId),h=document.getElementById(hdrId);
if(!b||!h)return;try{var s=localStorage.getItem(storageKey);
if(s==="1")setCollapsed(b,true);if(s==="0")setCollapsed(b,false);}catch(e){}
h.addEventListener("click",function(e){e.stopPropagation();
setCollapsed(b,!b.classList.contains("sat-about-collapsed"));});}
if(document.readyState==="loading"){document.addEventListener("DOMContentLoaded",init);}else{init();}})();</script>""")

    html_code = tpl.render(
        box_id=box_id, header_id=header_id,
        issue_number=issue_number, issue_url=issue_url,
        map_name=html.escape(map_name),
        created_date=created_date,
        links_html=links_html,
        collapsed_class=collapsed_class,
        map_dom_id=map_dom_id,
        top=top, left=left,
    )
    m.get_root().html.add_child(folium.Element(html_code))

# ========================
# Build combined map
# ========================
m = folium.Map(location=MAP_CENTER, zoom_start=MAP_ZOOM, tiles="OpenStreetMap")

add_commons_latest_layer(m, category="Stockholm_Archipelago_Trail", limit=100)
add_recent_edits_layer(m, days=1)
add_about_box(m, issue_number=221, map_name=OUTPUT_PREFIX)

folium.LayerControl(collapsed=False).add_to(m)

# --- Save outputs ---
os.makedirs("output", exist_ok=True)
ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
out_ts = os.path.join("output", f"{OUTPUT_PREFIX}_{ts}.html")
out_latest = os.path.join("output", f"{OUTPUT_PREFIX}_latest.html")

m.save(out_ts)
m.save(out_latest)

print("Klar:")
print("  ", out_ts)
print("  ", out_latest)


KeyError: 'label'

In [ ]:
end_time = time.time()
duration = end_time - start_time
print(f"Finished in {duration:.2f} seconds.")
